# Pneumonia detection - Clinical workflow integration test

In this notebook we will use our trained Pneumonia classification model to do an inference test on some test medical images to demonstrate a part of the clinical workflow integration.

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from keras.models import model_from_json
from skimage.transform import resize

Using TensorFlow backend.


In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    
    # check fields for required values
    if (ds.BodyPartExamined !='CHEST') | (ds.Modality !='DX') | (ds.PatientPosition not in ['PA', 'AP']):
        img=None
    else:
        img = ds.pixel_array

    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img): 
    
    img = img[:,:,None]
    img = resize(img, (1,224,224,3)) 
    img = np.asarray(img)
    proc_img = img / 255.0
    return proc_img


# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.load_weights(weight_path)
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    
    result = model.predict(img)
    prediction = result[0]
    if prediction > thresh:
        print('Pneumonia detected')
    else:
        print('No disease')
    return prediction 

In [3]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "my_model.json"
weight_path = "{}_my_model.best.hdf5".format('xray_class')

my_model = load_model(model_path, weight_path)
thresh = 0.15

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img)
    
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm ...
Pneumonia detected
[0.6071025]
Load file test2.dcm ...
Pneumonia detected
[0.60386366]
Load file test3.dcm ...
Pneumonia detected
[0.60456926]
Load file test4.dcm ...
Load file test5.dcm ...
Load file test6.dcm ...
